# Pré processamento dos conjuntos de dados

In [37]:
import pandas as pd
import os
from utils import get_workdir, bitarray_to_string, SEED
from binary_encoders import Thermometer, OneHot
from sklearn.model_selection import train_test_split

data = pd.read_csv(f"../dataset/internet/data.csv").drop("Unnamed: 0", axis="columns")
data

,id.orig_p,id.resp_p,proto,service,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type
0,38667,1883,tcp,mqtt,32.011598,9,5,3,3,0.281148,...,0.0,2.972918e+07,2.972918e+07,2.972918e+07,2.972918e+07,0.0,64240,26847,502,MQTT_Publish
1,51143,1883,tcp,mqtt,31.883584,9,5,3,3,0.282277,...,0.0,2.985528e+07,2.985528e+07,2.985528e+07,2.985528e+07,0.0,64240,26847,502,MQTT_Publish
2,44761,1883,tcp,mqtt,32.124053,9,5,3,3,0.280164,...,0.0,2.984215e+07,2.984215e+07,2.984215e+07,2.984215e+07,0.0,64240,26847,502,MQTT_Publish
3,60893,1883,tcp,mqtt,31.961063,9,5,3,3,0.281593,...,0.0,2.991377e+07,2.991377e+07,2.991377e+07,2.991377e+07,0.0,64240,26847,502,MQTT_Publish
4,51087,1883,tcp,mqtt,31.902362,9,5,3,3,0.282111,...,0.0,2.981470e+07,2.981470e+07,2.981470e+07,2.981470e+07,0.0,64240,26847,502,MQTT_Publish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123112,59247,63331,tcp,-,0.000006,1,1,0,0,167772.160000,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1024,0,1024,NMAP_XMAS_TREE_SCAN
123113,59247,64623,tcp,-,0.000007,1,1,0,0,144631.172414,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1024,0,1024,NMAP_XMAS_TREE_SCAN
123114,59247,64680,tcp,-,0.000006,1,1,0,0,167772.160000,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1024,0,1024,NMAP_XMAS_TREE_SCAN
123115,59247,65000,tcp,-,0.000006,1,1,0,0,167772.160000,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1024,0,1024,NMAP_XMAS_TREE_SCAN


In [38]:
data_desc = data.describe()
data_desc

,id.orig_p,id.resp_p,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,...,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size
count,123117.000000,123117.000000,123117.000000,123117.000000,123117.000000,123117.000000,123117.000000,1.231170e+05,1.231170e+05,1.231170e+05,...,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,1.231170e+05,123117.000000,123117.000000,123117.000000
mean,34639.258738,1014.305092,3.809566,2.268826,1.909509,1.471218,0.820260,3.518063e+05,3.517620e+05,7.035683e+05,...,1.481354e+05,2.353599e+04,1.616655e+06,1.701956e+06,3.517644e+06,1.664985e+06,4.550183e+04,6118.905123,2739.776018,751.647514
std,19070.620354,5256.371994,130.005408,22.336565,33.018311,19.635196,32.293948,3.707645e+05,3.708015e+05,7.415634e+05,...,1.613007e+06,1.477935e+06,8.809396e+06,9.252337e+06,1.229508e+08,9.007064e+06,1.091361e+06,18716.313861,10018.848534,6310.183843
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,17702.000000,21.000000,0.000001,1.000000,1.000000,1.000000,0.000000,7.454354e+01,7.288927e+01,1.490871e+02,...,9.536740e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
50%,37221.000000,21.000000,0.000004,1.000000,1.000000,1.000000,0.000000,2.467238e+05,2.467238e+05,4.934475e+05,...,4.053116e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
75%,50971.000000,21.000000,0.000005,1.000000,1.000000,1.000000,0.000000,5.242880e+05,5.242880e+05,1.048576e+06,...,5.006790e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
max,65535.000000,65389.000000,21728.335578,4345.000000,10112.000000,4345.000000,10105.000000,1.048576e+06,1.048576e+06,2.097152e+06,...,4.374931e+08,4.774862e+08,3.000000e+08,3.000000e+08,2.096777e+10,3.000000e+08,1.208029e+08,65535.000000,65535.000000,65535.000000


In [39]:
def create_column_encoder(data:pd.DataFrame)->dict:
    column_encoder = {}
    data_desc = data.describe()

    for column in data.columns:
        if column in data_desc.columns: # this is a numerical column
            column_encoder[column] = Thermometer(min=data_desc[column]["min"],max=data_desc[column]["max"],num_bits=32)
        else: # this is a categorical column
            column_encoder[column] = OneHot(list(data[column].unique()))
    
    return column_encoder

In [41]:
tgt = "Attack_type"

train_ft = data.drop(columns=["Attack_type"])
train_labels = data["Attack_type"]

column_encoder = create_column_encoder(train_ft)
column_encoder

{'id.orig_p': <binary_encoders.Thermometer at 0x7ec7cee00f50>,
 'id.resp_p': <binary_encoders.Thermometer at 0x7ec7cee03710>,
 'proto': <binary_encoders.OneHot at 0x7ec7cee01610>,
 'service': <binary_encoders.OneHot at 0x7ec7cee009e0>,
 'flow_duration': <binary_encoders.Thermometer at 0x7ec7cee00710>,
 'fwd_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee00680>,
 'bwd_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee02420>,
 'fwd_data_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee00380>,
 'bwd_data_pkts_tot': <binary_encoders.Thermometer at 0x7ec7cee00890>,
 'fwd_pkts_per_sec': <binary_encoders.Thermometer at 0x7ec7cee00c80>,
 'bwd_pkts_per_sec': <binary_encoders.Thermometer at 0x7ec7cee02300>,
 'flow_pkts_per_sec': <binary_encoders.Thermometer at 0x7ec7cee022a0>,
 'down_up_ratio': <binary_encoders.Thermometer at 0x7ec7cee01a00>,
 'fwd_header_size_tot': <binary_encoders.Thermometer at 0x7ec7cee00230>,
 'fwd_header_size_min': <binary_encoders.Thermometer at 0x7ec7cee01280>,

In [42]:
for column in column_encoder:
    train_ft[column] = [column_encoder[column].bitarray_encode(elem) for elem in train_ft[column]]
train_ft

/home/arthur/Documents/Trabs/GreenAI/wisard_grai/src/binary_encoders.py:18: RuntimeWarning: invalid value encountered in scalar divide
  bits_activated = int(math.ceil(((data - self.min)/ self.interval ) * self.num_bits))


ValueError: cannot convert float NaN to integer

In [ ]:
from bitarray import bitarray
def binarize_dataset(data, column_encoder):
    for column in column_encoder:
        train_ft[column] = [column_encoder[column].bitarray_encode(elem) for elem in train_ft[column]]
    train_mtx = []
    for idx, row in train_ft.iterrows():
        line = bitarray()
        for item in row:
            line.extend(item)
        train_mtx.append(line)
    return train_mtx

[bitarray('10011111111111111111000000000000000111111111111111110000000000000001110000000000000000000000000000011111100000000000000000000000000111110000000000000000000000000001111100000000000000000000000000011111000000000000000000000000000'), bitarray('10011111111111100000000000000000000111111111111000000000000000000001110000000000000000000000000000011100000000000000000000000000000111000000000000000000000000000001110000000000000000000000000000011100000000000000000000000000000'), bitarray('01011111111111111111111000000000000111111111111111111110000000000001111000000000000000000000000000011111111000000000000000000000000111111000000000000000000000000001111110000000000000000000000000011111110000000000000000000000000'), bitarray('10011111111111111110000000000000000111111111111111110000000000000001111000000000000000000000000000011111100000000000000000000000000111110000000000000000000000000001111100000000000000000000000000011111000000000000000000000000000'), bitarray('0011111111111110000000000